In [3]:
import os
import pandas as pd
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from utils import Utils as ut

path = ProjectPath()
setting = Setting()
dataloader = DataLoader()

cluster_path = path.univ2_cluster_path
cluster_file_template = "cluster_{}.csv" 
suspicious_file_template = "cluster_{}_suspicious_nodes.csv" 

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX
LOAD END NODES
LOAD CREATION INFO


/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token0"] = tokens["token0"].str.lower()
/home/dustin/Projects/SerialScammer/main/utils/DataLoader.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens["token1"] = tokens["token1"].str.lower()


LOAD RUG PULL INFO


In [5]:
group = 3
cluster_file = os.path.join(cluster_path, cluster_file_template.format(group))
suspicious_file = os.path.join(cluster_path, suspicious_file_template.format(group))
cluster = pd.read_csv(cluster_file)
suspicious_node = pd.read_csv(suspicious_file)
display(cluster.head())
display(suspicious_node.head())

,address,normal_txs_len,valid_neighbours_len,valid_neighbours,labels,path
0,0xedf646b301f0aab814af6cedf4f7a3cbff746be3,9,2,0xc99aa74da1c54e0be903f23ebac59f7ce6f29d93;0xe...,withdrawer;scammer,0xedf646b301f0aab814af6cedf4f7a3cbff746be3
1,0xc99aa74da1c54e0be903f23ebac59f7ce6f29d93,23,16,0x1ddf167010dd283f31325659c112dafb6f8b849b;0x1...,withdrawer;washtrader,0xedf646b301f0aab814af6cedf4f7a3cbff746be3;0xc...
2,0x1ddf167010dd283f31325659c112dafb6f8b849b,4,2,0xc99aa74da1c54e0be903f23ebac59f7ce6f29d93;0xe...,washtrader,0xedf646b301f0aab814af6cedf4f7a3cbff746be3;0xc...
3,0x1bcff207d1a8bb476429bf42570f9f5de4a28c28,4,2,0xc99aa74da1c54e0be903f23ebac59f7ce6f29d93;0xe...,washtrader,0xedf646b301f0aab814af6cedf4f7a3cbff746be3;0xc...
4,0xa3d6a8789cb8ed5eaea0fb9b25a2d435f5fda17c,4,2,0xc99aa74da1c54e0be903f23ebac59f7ce6f29d93;0xe...,washtrader,0xedf646b301f0aab814af6cedf4f7a3cbff746be3;0xc...


,address,valid_neighbours,normal_txs_len,labels
0,0xcc3337f6fe821a7a654552395e128e2c4fcabe31,127,519,depositor;big_connector;withdrawer
1,0x68bec4525eb557f48c30eefec249b1d85706cf0c,90,1103,withdrawer;washtrader;big_connector
2,0xb2cd290b0f0ddfd07fe19f8f5aa1474a51caf922,308,4895,depositor;washtrader;big_connector;withdrawer
3,0x167c6a6948606e410cd79baac3be52d949cbcd3c,135,2364,withdrawer;washtrader;big_connector
4,0x4b2b31b7a32a3d3bb894effc31b16abd400bd258,77,969,withdrawer;washtrader;scammer;big_connector
